In [3]:
# taken partially from tutorial at: https://stmorse.github.io/journal/espn-fantasy-python.html
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import requests

league1 = 380134
league2 = 1553391
leagues=[league1, league2]

weeks = 6

scores={}

for i in range(len(leagues)):
    scores[i] = {}
for week in range(1, weeks+1):
    for league in range(len(leagues)):
        r = requests.get('http://games.espn.com/ffl/api/v2/scoreboard', 
                         params={'leagueId': leagues[league], 'seasonId': 2018, 'matchupPeriodId': week})
        scores[league][week] = r.json()

df = []
dfRecord = []
for league in range(len(leagues)):
    for week in scores[league]:
        temp = scores[league][week]['scoreboard']['matchups']
        for match in temp:
            
            df.append([week, 
                       match['teams'][0]['team']['teamAbbrev'],
                       match['teams'][1]['team']['teamAbbrev'],
                       match['teams'][0]['score'],
                       match['teams'][1]['score']
                      ])


df = pd.DataFrame(df, columns=['Week', 'HomeAbbrev', 'AwayAbbrev', 'HomeScore', 'AwayScore'])
df.head()
df = (df[['Week', 'HomeAbbrev', 'HomeScore']]
      .rename(columns={'HomeAbbrev': 'Abbrev', 'HomeScore': 'Score'})
      .append(df[['Week', 'AwayAbbrev', 'AwayScore']]
             .rename(columns={'AwayAbbrev': 'Abbrev', 'AwayScore': 'Score'}))
     )

# now adding the record:

for league in range(len(leagues)):
    temp2=scores[league][weeks]['scoreboard']['matchups']
    for match in temp2:
        for team in match['teams']:
#         append "points for" for debugging purposes
            dfRecord.append([
                            team['team']['teamAbbrev'],
                            team['team']['record']['pointsFor'],
                            team['team']['record']['pointsAgainst'],
                            team['team']['record']['overallWins'],
                            team['team']['record']['overallLosses'],
                            team['team']['record']['overallTies']
                            ])
dfRecord = pd.DataFrame(dfRecord, columns=["Abbrev", "Points For", "Points Against", "Overall Wins","Overall Losses", "Overall Ties"])
dfRecord = dfRecord.set_index('Abbrev')
df1 = df.groupby("Abbrev")["Score"].describe().drop("count", axis=1)
s1 = df.groupby("Abbrev")['Score'].sum()
finalDF = df1.join(dfRecord)
finalDF = finalDF.sort_values(by=['Overall Wins','Points For'], ascending=[False,False])
finalDF = finalDF.reset_index()
finalDF.index +=1
display(finalDF)

,Abbrev,mean,std,min,25%,50%,75%,max,Points For,Points Against,Overall Wins,Overall Losses,Overall Ties
1,LEE,101.250000,17.888646,78.62,90.445,99.40,110.035,129.04,607.50,494.46,6,0,0
2,LURI,120.366667,21.999493,93.26,105.830,122.54,126.950,155.18,722.20,545.08,5,1,0
3,FITZ,113.320000,12.904275,97.00,108.005,111.95,115.250,135.98,679.92,487.18,5,1,0
4,AUTO,109.353333,19.831561,73.80,103.430,117.58,120.150,127.44,656.12,607.64,5,1,0
5,NATH,86.996667,15.013426,67.90,75.940,88.06,99.940,102.16,521.98,480.40,5,1,0
6,PETR,115.490000,41.295710,58.32,101.445,112.90,122.750,185.16,692.94,591.30,4,2,0
7,HARN,111.593333,16.463636,83.36,106.185,114.89,121.165,129.88,669.56,665.56,4,2,0
8,AQP,110.740000,11.180447,100.74,102.025,107.15,117.375,128.30,664.44,521.62,4,2,0
9,DREW,106.270000,18.773666,78.48,94.075,110.25,120.050,126.64,637.62,534.84,4,2,0
10,MACK,104.726667,20.898236,86.06,89.085,97.23,117.675,136.98,628.36,550.20,4,2,0
